In [1]:
from vpython import *
import sys

# Parameters for the lattice
rows, cols, layers = 5, 5, 20  # Number of balls along x, y, z

#parameters for material
mass = 9.27e-26  # Mass of each atom
density = (7800) * 7
molar_mass = 0.05585
youngs_modulus = (211e9)

avo = 6.02e23
spacing = (molar_mass/(density*avo))**(1/3)  
radius = spacing*0.2  # Radius of the balls
spring_thickness = 0.1*286.65e-12  # Thickness of the springs
spring_constant = spacing*youngs_modulus  # Spring k
theoretical_speed = sqrt(youngs_modulus*spacing**3/(mass))  # theoretical wave speed
dt = ((layers-1)*spacing/theoretical_speed)/1000
driving_force_amplitude = 0.2*spacing
driving_force_frequency = 1e10
t = 0

# Create a scene 
scene = canvas(title="Wave Propagation in Crystal", background=color.white)
scene.range = spacing*layers*5
scene.camera.pos = vector((rows+layers)*spacing, 0, spacing*layers/2)  # Move camera farther away
scene.camera.axis = vector(-(rows+layers)*spacing, 0, -spacing*layers/2)

graph = graph(title="End of lattice position", xtitle="Time (s)", ytitle="position relative to initial", xmin = 0, 
              xmax = ((layers-1)*spacing/theoretical_speed)*2,
            ymax  = 4*driving_force_amplitude, ymin = -4*driving_force_amplitude)
draw = gcurve(color = color.blue)
# Store the balls and springs
balls = []
springs = []

# Create the lattice of balls and assign properties
for x in range(rows):
    for y in range(cols):
        for z in range(layers):
            ball = sphere(pos=vector(x * spacing, y * spacing, z * spacing), 
                        radius=radius, 
                        color=color.red)
            ball.velocity = vector(0, 0, 0)
            ball.force = vector(0, 0, 0)
            ball.initial_z = z * spacing  # Store initial z position
            balls.append(ball)

# Create the springs (connect adjacent balls)
connections = []
tolerance = 1e-10
for ball in balls:
    for dx, dy, dz in [(1, 0, 0), (0, 1, 0), (0, 0, 1)]:
        neighbor_pos = ball.pos + vector(dx * spacing, dy * spacing, dz * spacing)
        for neighbor in balls:
            if mag(neighbor.pos - neighbor_pos) < tolerance:  # Use a small tolerance
                spring = helix(pos=ball.pos, 
                               axis=neighbor.pos - ball.pos, 
                               radius=spring_thickness, 
                               color=color.green)
                springs.append(spring)
                connections.append((ball, neighbor))



for ball in balls:
    ball.momentum = vector(0,0,0)

# Variables for wave detection
wave_threshold = driving_force_amplitude*0.5
wait = 0
first_peak_time = 0
# Simulation loop
while True:
    rate(1000) 
    
    
    # Reset forces
    for ball in balls:
        ball.force = vector(0, 0, 0)
       
        
    # Calculate spring forces
    for ball1, ball2 in connections:
        displacement = ball2.pos - ball1.pos
        distance = mag(displacement)
        direction = norm(displacement)
        stretch = distance - spacing
        force = spring_constant * stretch * direction
        ball1.force += force
        ball2.force -= force
        
     #Add driving force for first layer
    for ball in balls:
        if abs(ball.initial_z - 0) < 1e-10:
            ball.pos.z =  driving_force_amplitude*cos(driving_force_frequency*t/(2*pi))
        else:
            ball.momentum += ball.force*dt
            ball.velocity = ball.momentum / mass
            ball.pos += ball.velocity * dt
    # Update velocities and positions
    #for ball in balls:
        
            
            # Check for wave arrival at top
        if ball.pos.z > ((layers-1)*spacing + wave_threshold) and first_peak_time < 1e-15:
            first_peak_time = t
            measured_speed = (layers-1)*spacing/t
            print("\nWave Propagation Analysis:")
            print(f"Time to reach top = {t:.2e} s")
            print(f"Driving frequency = {driving_force_frequency} Hz")
            print(f"Driving amplitude = {driving_force_amplitude:.2e} m")
            print(f"Measured wave speed = {measured_speed:.2e} m/s")
            print(f"Theoretical wave speed = {theoretical_speed:.2e} m/s")
            print(f"Speed ratio (measured/theoretical) = {measured_speed/theoretical_speed:.3f}")
            wait = t
            
    if wait > 0:
        if t > 20*wait :
            sys.exit()
            
    record = 0
    for ball in balls:
        if abs(ball.initial_z - (layers-1)*spacing) < tolerance and record == 0:
            draw.plot(pos = (t, ball.pos.z - ball.initial_z))
            record =1
        
    # Update spring positions
    for spring, (ball1, ball2) in zip(springs, connections):
        spring.pos = ball1.pos
        spring.axis = ball2.pos - ball1.pos
        
    t += dt

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Wave Propagation Analysis:
Time to reach top = 1.15e-12 s
Driving frequency = 10000000000.0 Hz
Driving amplitude = 2.39e-11 m
Measured wave speed = 1.96e+03 m/s
Theoretical wave speed = 1.97e+03 m/s
Speed ratio (measured/theoretical) = 0.999


SystemExit: 

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
